In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime
from sklearn.linear_model import LassoCV, LinearRegression
from hmmlearn import hmm
import time

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
# Input raw data (two first lines for Corbin and Young)

input_df = pd.read_csv('~/Desktop/Corbin SBU/AMS 520/Project/BofA Projects Data/EOD_20210908.csv',
# input_df = pd.read_csv('D:\\EOD\\EOD_20210908.csv',
                       header = None,
                       names = ['Ticker', # Label columns
                                'Date',
                                'Open',
                                'High',
                                'Low',
                                'Close',
                                'Volume',
                                'Dividend',
                                'Stock_split',
                                'Adj_open',
                                'Adj_high',
                                'Adj_low',
                                'Adj_close',
                                'Adj_volume'])

In [3]:
def asset_data(input_df,
               ticker='SPY',
               lookback=260):
    """
    Function purpose
    -------------------------------------
    Generate asset data
    
    Parameters
    ----------
    input_df : dataframe
        Historical data for asset, must include the following columns:
            - 'Ticker'
            - 'Date'
            - 'Adj_open'
            - 'Adj_high'
            - 'Adj_low'
            - 'Adj_close'
    ticker : str
        Selected asset
    lookback : int
        Lookback period (days)
    
    Return
    ------
    Data for selected asset
    """
    
    # Select which index to use for analysis
    data = input_df.loc[input_df['Ticker'] == ticker]
    data.reset_index(inplace=True, drop=True)
    data.set_index('Date', inplace=True)
    
    # Compute daily percentage returns
#     returns = 100*data['Adj_close'].pct_change(1)
    returns = data['Adj_close'].pct_change(1)
    data['Returns'] = returns
    
    # Compute daily volatility
    volatility = get_ohlc_vol(data,
                              lookback=Neff)
    data['Volatility'] = volatility
    
    return data

In [4]:
# Proposed Idea: Create a HMM for the recent Neff days, and for all days after the Neff'th day
# predict which state we are currently in based on the Neff recent days.
# Be fully invested if in a positive market, fully divested in a negative market, but cash position grows from investment in Treasury.

In [5]:
def dynamicHMM(input_df, ticker='SPY', regression=True, lassoCV=True, cv=5, lookback=260, reg_lookback=21):
    """
    Function purpose
    -------------------------------------
    Generate portfolio returns for a regime-switching model
    
    
    Parameters
    ----------
    input_df : dataframe
        Historical data for asset, must include the following columns:
            - 'Ticker'
            - 'Date'
            - 'Adj_open'
            - 'Adj_high'
            - 'Adj_low'
            - 'Adj_close'
    ticker : str
        Selected asset
    regression : True/False
        Select True to include a regression in the model
    lassoCV : True/False
        Select True to perform a LassoCV regression
        Select False to perform a Linear regression
    lookback : int
        Lookback period (days)
    reg_lookback : int
        Lookback period (days) for regression
    
    Return
    ------
    Array of states over time (i.e. list of 0s and 1s for negative states and positive states, respectively)
    """

    # Generate stock data
    data = asset_data(input_df,
                      ticker=ticker,
                      lookback=lookback)
    # Adjust volatility input for nuances with GaussianHMM function
    if (not lassoCV) or (not regression):
        data['Volatility'] *= 100

    if regression:
        # Sources: stlouisfed.org, yahoo finance
        # Helpful guids: https://rdrr.io/github/AndreMikulec/econModel/src/R/StressIndex.R
        # Regression to use to predict index

        # Input regression data (two first lines for Corbin and Young)
        reg_df = pd.read_excel('~/Desktop/Corbin SBU/AMS 520/Project/Project Code/GitHub/Regime-Detection-HMM/Regression_Variables.xlsx')
        # reg_df = pd.read_excel('C:\\Users\\ryans\\Desktop\\AMS\\520\\Regression_Variables.xlsx')

        # Perform LassoCV regression to predict day-ahead asset returns
        reg_df.rename(columns={'observation_date': 'Date'}, inplace=True)
        reg_df.set_index('Date', inplace=True)

        # Merge regression data with asset data
        merged_data = data.join(reg_df, how='outer').loc[data.index[0]:data.index[-1],].dropna(subset=['Ticker'])
        merged_data.drop(columns=['Ticker',
                              'Open',
                              'High',
                              'Low',
                              'Close',
                              'Volume',
                              'Dividend',
                              'Stock_split',
                              'Adj_open',
                              'Adj_high',
                              'Adj_low',
                              'Adj_close',
                              'Adj_volume',
                              'Volatility'],
                    inplace=True)

        # Fill all missing data points with the most recently available value
        merged_data.fillna(method = 'ffill', inplace = True)

        # Perform a regression to predict day-ahead returns
        if lassoCV:
            regression_pred = LassoCVReg(merged_data,
                                         cv,
                                         lookback=reg_lookback)
        else:
            regression_pred = LinReg(merged_data,
                                     lookback=reg_lookback)
            
        # Store predicted returns in a dataframe
        reg_predicted_returns = pd.DataFrame(index=data.index[-len(regression_pred):],
                                             data=regression_pred,
                                             columns=['Regression_Prediction'])
        
        # Merge predicted returns with daily SPY data
        data = data.join(reg_predicted_returns, how='outer')
        print(data)
    else:
        reg_df = None
        
    # Fit the HMM for each time step
    current_state = fitHMM(data,
                           regression=regression,
                           lookback=lookback)

    return data, reg_df, current_state


In [6]:
def get_ohlc_vol(data,
                 lookback=260):
    """
    Function purpose
    -------------------------------------
    Create a series of OHLC volatilities based upon historical asset data
    
    Parameters
    ----------
    data : dataframe
        Historical data for asset, must include the following columns:
            - 'Adj_open'
            - 'Adj_high'
            - 'Adj_low'
            - 'Adj_close'
    lookback : int
        Lookback period (days)
    
    Return
    ------
    Volatility at each time step
    
    Source
    ------
    https://dynamiproject.files.wordpress.com/2016/01/measuring_historic_volatility.pdf
    Yang-Zhang Method
    """
    
    o = data.Adj_open
    h = data.Adj_high
    l = data.Adj_low
    c = data.Adj_close
    
    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    
    # close_vol = pd.rolling_sum(cc_sq, window=lookback) * (1.0 / (lookback - 1.0))
    close_vol = cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))

    # open_vol = pd.rolling_sum(oc_sq, window=lookback) * (1.0 / (lookback - 1.0))
    open_vol = oc_sq.rolling(lookback).sum()  * (1.0 / (lookback - 1.0))
    
    # window_rs = pd.rolling_sum(rs, window=lookback) * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    
    volatility = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    volatility[:lookback-1] = 0.0
    
    return volatility

In [7]:
def LassoCVReg(merged_data,
               cv=5,
               lookback=21):
    """
    Function purpose
    -------------------------------------
    Perform a Lasso CV regression to predict day-ahead asset returns
    
    Parameters
    ----------
    merged_data : dataframe
        Historical data for asset and regression inputs
    cv : int
        Number of cross-validation folds
    lookback : int
        Lookback period (days)
    
    Return
    ------
    Predicted day-ahead asset returns
    """

    regression_pred = np.array([]) # Store predicted asset return values

    for i in range(len(merged_data) - lookback):

        print(f'LassoCV Regression: {i}')

        # Check if and which independent variables have NA values
        na_check = pd.isna(merged_data.iloc[i:lookback + i, 1:])
        ind_var_available = np.array([])

        # Include independent variables only if they have sufficient data, i.e. lookback number of days at each time step
        for j, k in enumerate(na_check):
            if not any(na_check.loc[:,k]):
                ind_var_available = np.append(ind_var_available, j+1)

        ind_var_available = ind_var_available.astype(np.int)

        X = merged_data.iloc[i:lookback + i, ind_var_available].to_numpy()
        y = merged_data['Returns'][1 + i:lookback + 1 + i].to_numpy()

        lassoCV = LassoCV(cv=cv)
        lassoCV.fit(X, y)
        regression_pred = np.append(regression_pred, lassoCV.predict([X[-1]])[0])
        
    return regression_pred


In [8]:
def LinReg(merged_data,
           lookback=21):
    """
    Function purpose
    -------------------------------------
    Perform a Linear regression to predict day-ahead asset returns
    
    Parameters
    ----------
    merged_data : dataframe
        Historical data for asset and regression inputs
    lookback : int
        Lookback period (days)
    
    Return
    ------
    Predicted day-ahead asset returns
    """

    regression_pred = np.array([]) # Store predicted asset return values

    for i in range(len(merged_data) - lookback):

        print(f'Linear Regression: {i}')

        # Check if and which independent variables have NA values
        na_check = pd.isna(merged_data.iloc[i:lookback + i, 1:])
        ind_var_available = np.array([])

        # Include independent variables only if they have sufficient data, i.e. lookback number of days at each time step
        for j, k in enumerate(na_check):
            if not any(na_check.loc[:,k]):
                ind_var_available = np.append(ind_var_available, j+1)

        ind_var_available = ind_var_available.astype(np.int)

        X = merged_data.iloc[i:lookback + i, ind_var_available].to_numpy()
        y = merged_data['Returns'][1 + i:lookback + 1 + i].to_numpy()

        linReg = LinearRegression()
        linReg.fit(X, y)
        regression_pred = np.append(regression_pred, linReg.predict([X[-1]])[0])

    return regression_pred


In [9]:
def fitHMM(data,
           regression=True,
           lookback=260):
    """
    Function purpose
    -------------------------------------
    Perform an hmm.GaussianHMM (Hidden Markov Model) fit for each time step using a rolling lookback window
    
    Parameters
    ----------
    data : dataframe
        Historical data for asset and regression inputs, must include the following columns:
            - 'Returns'
            - 'Volatility'
            - 'Regression_Prediction' (if regression=True)
    regression : True/False
        Select True to include a regression in the HMM        
    lookback : int
        Lookback period (days)
    
    Return
    ------
    Array of states over time (i.e. list of 0s and 1s for negative states and positive states, respectively)
    """

    current_state = np.array([]) # Initialize an array to track the current regimes

    # Exclude first (to calculate trailing volatility) and
    # second (to have a full set of observations to fit a HMM) lookback number of days of observations
    for i in range(len(data) - 2*lookback):

        print(f'HMM fit: {i}')

        # Initialize a Gaussian HMM
        model = hmm.GaussianHMM(n_components = states, covariance_type=covariance_type, n_iter = max_iterations)

        # Pull lookback days of observations of returns, volatility, and day-ahead predicted returns (if included)
        if regression:
            observations = data.iloc[i + lookback:i + 2*lookback,:].loc[:,['Returns', 'Volatility', 'Regression_Prediction']].to_numpy()
        else:
            observations = data.iloc[i + lookback:i + 2*lookback,:].loc[:,['Returns', 'Volatility']].to_numpy()

        model.fit(observations) # Fit the model to the observations

        # Model randomly allocates a '0' or a '1' to a state, so check which state has the higher mean return
        if model.means_[0,0] > model.means_[1,0]:
            positive_state = 0
        else:
            positive_state = 1

        predictions = model.predict(observations) # Predict the state for each observation

        if positive_state == 1: # If the state with the higer mean return is state 1, do nothing, if not...
            pass
        else: # Switch the predicted regimes to ensure state 1 is always the regime with a greater mean return
            zeros = np.where(predictions == 0)
            ones = np.where(predictions == 1)
            predictions[zeros] = 1
            predictions[ones] = 0

        # Append the current state to the regime tracker
        current_state = np.append(current_state,predictions[-1])

    # Switch values from type float to type int for later calculations
    current_state = current_state.astype(np.int64)
    
    return current_state

In [10]:
def regime_switching_stats(current_state):
    """
    Function purpose
    -------------------------------------
    Compute regime switching metrics
    
    Parameters
    ----------
    current_state : array
        Current market state
    
    Return
    ------
    Output regime switching information
    """

    # Number of regime change in the data sets
    total_regime_switches = sum(np.abs(current_state[1:] - current_state[:-1]))

    years_of_data = len(current_state) / trading_days
    avg_regime_changes_per_year = total_regime_switches / years_of_data

    print(f'Total regime switches: {total_regime_switches}')
    print(f'Average number of regime switches per year: {avg_regime_changes_per_year:.01f}')

In [11]:
def regime_switching_plot(data,
                          current_state,
                          lookback=260):
    """
    Function purpose
    -------------------------------------
    Produce a plot of regime changes
    
    Parameters
    ----------
    data : dataframe
        Historical data for asset, must have 'Date' as the index
    current_state : array
        Current market state
    lookback : int
        Lookback period (days)
        
    Return
    ------
    Output plot of regime changes
    """

    dates = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in data.index[2*lookback:]]

    fig, ax = plt.subplots(figsize=(18,3))

    formatter = mdates.DateFormatter("%Y")

    ax.xaxis.set_major_formatter(formatter)

    fmt_half_year = mdates.MonthLocator(interval=24)
    ax.xaxis.set_major_locator(fmt_half_year)

    ax.plot(dates, current_state, '.', color = 'royalblue');

In [12]:
def regime_state_stats(data,
                       current_state,
                       lookback=260):
    """
    Function purpose
    -------------------------------------
    Compute basic stats for each state
    
    Parameters
    ----------
    data : dataframe
        Historical data for asset, must have 'Date' as the index
    current_state : array
        Current market state
    lookback : int
        Lookback period (days)
        
    Return
    ------
    Output regime statistics
    """

    state_0 = np.where(current_state == 0)[0] + 2*lookback
    state_1 = np.where(current_state == 1)[0] + 2*lookback

    print(f'Number of occurences of state 0: {len(state_0)}')
    print(f'Mean return of state 0: {data.iloc[state_0].Returns.mean():.03f}')
    print(f'Volatility of state 0: {data.iloc[state_0].Returns.std():.03f}')
    print('\n')
    print(f'Number of occurences of state 1: {len(state_1)}')
    print(f'Mean return of state 1: {data.iloc[state_1].Returns.mean():.03f}')
    print(f'Volatility of state 1: {data.iloc[state_1].Returns.std():.03f}')

In [13]:
def asset_switching_plot(data,
                         current_state,
                         lookback=260):
    """
    Function purpose
    -------------------------------------
    Produce a plot of the asset, colored by regime, over time
    
    Parameters
    ----------
    data : dataframe
        Historical data for asset, must have 'Date' as the index
    current_state : array
        Current market state
    lookback : int
        Lookback period (days)
        
    Return
    ------
    Output plot of asset dynamics
    """

    plot = sns.relplot(x = range(0,len(current_state)),
                       y = "Adj_close",
                       data = data[2*lookback:],
                       hue = current_state,
                       linewidth = 0,
                       palette = "Set2",
                       s = 10);

    plot.fig.set_size_inches(18,10)

In [14]:
def portfolio_performance(data,
                          current_state,
                          lookback=260):
    """
    Function purpose
    -------------------------------------
    Compute portfolio performance
    
    Parameters
    ----------
    data : dataframe
        Historical data for asset, must have 'Date' as the index
    current_state : array
        Current market state
    lookback : int
        Lookback period (days)
        
    Return
    ------
    Array of closing portfolio value
    """

    reg_df = pd.read_excel('~/Desktop/Corbin SBU/AMS 520/Project/Project Code/GitHub/Regime-Detection-HMM/Regression_Variables.xlsx')
    reg_df.set_index('observation_date', inplace=True)
    
    # Plan to invest in treasury if divested (state 0)
    daily_10_year_treas = pd.DataFrame(reg_df.loc[:,'Var3_10_Yr_Treas'], index=reg_df.index)**(1/252) - 1
    daily_10_year_treas.fillna(method = 'ffill', inplace = True)
    
    # Merge treasury with data
    data = data.join(daily_10_year_treas, how='outer').loc[data.index[0]:data.index[-1],:].dropna(subset=['Ticker'])
    
    # Compute portfolio returns
    portfolio_returns = current_state * data.Returns[2*lookback:] + (1 - current_state) * data.Var3_10_Yr_Treas[2*lookback:] / 100 + 1
    
    # Compute portfolio performance
    portfolio_perf = data.Adj_close[2*Neff-1] * np.cumprod(portfolio_returns)
    
    return portfolio_perf


In [15]:
def asset_switching_and_portfolio_plot(data,
                                       current_state,
                                       portfolio_performance,
                                       lookback=260):
    """
    Function purpose
    -------------------------------------
    Produce a plot of the asset, colored by regime, and the our portfolio over time
    
    Parameters
    ----------
    data : dataframe
        Historical data for asset, must have 'Date' as the index
    current_state : array
        Current market state
    portfolio_performance : array
        Closing value of portfolio
    lookback : int
        Lookback period (days)
        
    Return
    ------
    Output plot of asset and portfolio dynamics
    """

    plot = sns.relplot(x = range(0,len(current_state)),
                       y = "Adj_close",
                       data = data[2*Neff:],
                       hue = current_state,
                       linewidth = 0,
                       palette = "Set2",
                       s = 10);

    plt.plot(range(0,len(current_state)), portfolio_performance, color='royalblue')
    plt.legend(['HMM Portfolio','SPY (state 1)', 'SPY (state 0)'])

    plot.fig.set_size_inches(14,7)

In [16]:
def sharpe_ratio(portfolio_perf,
                 trading_days=252):
    """
    Function purpose
    -------------------------------------
    Computes the annualized sharpe ratio of the portfolio, assumes no risk-free rate
    
    Parameters
    ----------
    portfolio_perf : array
        Closing value of portfolio
    trading_days : int
        Number of trading days in period (daily, monthly, etc.)
        
    Return
    ------
    Annualized sharpe ratio
    """    

    # Compute portfolio daily returns
    portfolio_perf_returns = portfolio_perf.pct_change(1)
    
    portfolio_sharpe = portfolio_perf_returns.mean() / portfolio_perf_returns.std() * np.sqrt(trading_days)
    
    return portfolio_sharpe

In [17]:
def save_portfolio_performance(portfolio_perf,
                               Neff=260,
                               regression=True):
    """
    Function purpose
    -------------------------------------
    Save portfolio performance given different inputs
    
    Parameters
    ----------
    portfolio_perf : array
        Closing value of portfolio
    Neff : int
        Lookback period (days)
    regression : True/False
        Select True to include a regression in the model
        
    Return
    ------
    Saves .csv file to specified folder
    """
    
    Neff_str = str(Neff)
    if lassoCV and regression:
        reg_str = 'lassoCV'
    elif (not lassoCV) and regression:
        reg_str = 'LinReg'
    else:
        reg_str = 'NoReg'

    np.savetxt('Portfolio_Performances/portfolio' + '_' + Neff_str + '_' + reg_str + '.csv',
#    np.savetxt('Portfolio_Performances/portfolio' + '_' + Neff_str + '_' + reg_str + '2.csv',
               portfolio_perf,
               delimiter=',')

In [18]:
# Data inputs
ticker = 'SPY' # Selected asset

# Model inputs
trading_days = 252 # Approximately 252 trading days in a year
Neff = 260 # Lookback period

# Regression inputs
regression = True
reg_lookback = 21 # Representing trading days in a month
lassoCV = True
cv = 5 # Number of CV folds

# GaussianHMM inputs
states = 2 # Number of states
covariance_type = 'full' # Covariance method
max_iterations = 100 # For EM algorithm

In [19]:
def compute_portfolio_save_performance(input_df=input_df,
                                       ticker=ticker,
                                       regression=regression,
                                       lassoCV=lassoCV,
                                       cv=cv,
                                       lookback=Neff,
                                       reg_lookback=reg_lookback):

    start = time.time()

    data, reg_df, current_state = dynamicHMM(input_df,
                                             ticker=ticker,
                                             regression=regression,
                                             lassoCV=lassoCV,
                                             cv=cv,
                                             lookback=Neff,
                                             reg_lookback=reg_lookback)
    
    regime_switching_stats(current_state)

    regime_switching_plot(data,
                          current_state,
                          lookback=Neff)

    regime_state_stats(data,
                       current_state,
                       lookback=Neff)

    asset_switching_plot(data,
                         current_state,
                         lookback=Neff)

    portfolio_perf = portfolio_performance(data,
                                           current_state,
                                           lookback=Neff)

    asset_switching_and_portfolio_plot(data,
                                       current_state,
                                       portfolio_performance=portfolio_perf,
                                       lookback=Neff)

    portfolio_sharpe = sharpe_ratio(portfolio_perf=portfolio_perf,
                                    trading_days=trading_days)
    print(f'Sharpe Ratio: {portfolio_sharpe:.3f}')
    
#     save_portfolio_performance(portfolio_perf=portfolio_perf,
#                                Neff=Neff,
#                                regression=regression)

    end = time.time()
    print(f'Computation time (seconds): {end - start:0f}')
    
    return data, reg_df, current_state, portfolio_perf, portfolio_sharpe

In [20]:
data, reg_df, current_state, portfolio_perf, portfolio_sharpe = compute_portfolio_save_performance(
    input_df=input_df,
    ticker=ticker,
    regression=regression,
    lassoCV=lassoCV,
    cv=cv,
    lookback=Neff,
    reg_lookback=reg_lookback)

LassoCV Regression: 0
LassoCV Regression: 1
LassoCV Regression: 2
LassoCV Regression: 3
LassoCV Regression: 4
LassoCV Regression: 5
LassoCV Regression: 6
LassoCV Regression: 7
LassoCV Regression: 8
LassoCV Regression: 9
LassoCV Regression: 10
LassoCV Regression: 11
LassoCV Regression: 12
LassoCV Regression: 13
LassoCV Regression: 14
LassoCV Regression: 15
LassoCV Regression: 16
LassoCV Regression: 17
LassoCV Regression: 18
LassoCV Regression: 19
LassoCV Regression: 20
LassoCV Regression: 21
LassoCV Regression: 22
LassoCV Regression: 23
LassoCV Regression: 24
LassoCV Regression: 25
LassoCV Regression: 26
LassoCV Regression: 27
LassoCV Regression: 28
LassoCV Regression: 29
LassoCV Regression: 30
LassoCV Regression: 31
LassoCV Regression: 32
LassoCV Regression: 33
LassoCV Regression: 34
LassoCV Regression: 35
LassoCV Regression: 36
LassoCV Regression: 37
LassoCV Regression: 38
LassoCV Regression: 39
LassoCV Regression: 40
LassoCV Regression: 41
LassoCV Regression: 42
LassoCV Regression: 4

LassoCV Regression: 346
LassoCV Regression: 347
LassoCV Regression: 348
LassoCV Regression: 349
LassoCV Regression: 350
LassoCV Regression: 351
LassoCV Regression: 352
LassoCV Regression: 353
LassoCV Regression: 354
LassoCV Regression: 355
LassoCV Regression: 356
LassoCV Regression: 357
LassoCV Regression: 358
LassoCV Regression: 359
LassoCV Regression: 360
LassoCV Regression: 361
LassoCV Regression: 362
LassoCV Regression: 363
LassoCV Regression: 364
LassoCV Regression: 365
LassoCV Regression: 366
LassoCV Regression: 367
LassoCV Regression: 368
LassoCV Regression: 369
LassoCV Regression: 370
LassoCV Regression: 371
LassoCV Regression: 372
LassoCV Regression: 373
LassoCV Regression: 374
LassoCV Regression: 375
LassoCV Regression: 376
LassoCV Regression: 377
LassoCV Regression: 378
LassoCV Regression: 379
LassoCV Regression: 380
LassoCV Regression: 381
LassoCV Regression: 382
LassoCV Regression: 383
LassoCV Regression: 384
LassoCV Regression: 385
LassoCV Regression: 386
LassoCV Regressi

LassoCV Regression: 688
LassoCV Regression: 689
LassoCV Regression: 690
LassoCV Regression: 691
LassoCV Regression: 692
LassoCV Regression: 693
LassoCV Regression: 694
LassoCV Regression: 695
LassoCV Regression: 696
LassoCV Regression: 697
LassoCV Regression: 698
LassoCV Regression: 699
LassoCV Regression: 700
LassoCV Regression: 701
LassoCV Regression: 702
LassoCV Regression: 703
LassoCV Regression: 704
LassoCV Regression: 705
LassoCV Regression: 706
LassoCV Regression: 707
LassoCV Regression: 708
LassoCV Regression: 709
LassoCV Regression: 710
LassoCV Regression: 711
LassoCV Regression: 712
LassoCV Regression: 713
LassoCV Regression: 714
LassoCV Regression: 715
LassoCV Regression: 716
LassoCV Regression: 717
LassoCV Regression: 718
LassoCV Regression: 719
LassoCV Regression: 720
LassoCV Regression: 721
LassoCV Regression: 722
LassoCV Regression: 723
LassoCV Regression: 724
LassoCV Regression: 725
LassoCV Regression: 726
LassoCV Regression: 727
LassoCV Regression: 728
LassoCV Regressi

LassoCV Regression: 1029
LassoCV Regression: 1030
LassoCV Regression: 1031
LassoCV Regression: 1032
LassoCV Regression: 1033
LassoCV Regression: 1034
LassoCV Regression: 1035
LassoCV Regression: 1036
LassoCV Regression: 1037
LassoCV Regression: 1038
LassoCV Regression: 1039
LassoCV Regression: 1040
LassoCV Regression: 1041
LassoCV Regression: 1042
LassoCV Regression: 1043
LassoCV Regression: 1044
LassoCV Regression: 1045
LassoCV Regression: 1046
LassoCV Regression: 1047
LassoCV Regression: 1048
LassoCV Regression: 1049
LassoCV Regression: 1050
LassoCV Regression: 1051
LassoCV Regression: 1052
LassoCV Regression: 1053
LassoCV Regression: 1054
LassoCV Regression: 1055
LassoCV Regression: 1056
LassoCV Regression: 1057
LassoCV Regression: 1058
LassoCV Regression: 1059
LassoCV Regression: 1060
LassoCV Regression: 1061
LassoCV Regression: 1062
LassoCV Regression: 1063
LassoCV Regression: 1064
LassoCV Regression: 1065
LassoCV Regression: 1066
LassoCV Regression: 1067
LassoCV Regression: 1068


LassoCV Regression: 1359
LassoCV Regression: 1360
LassoCV Regression: 1361
LassoCV Regression: 1362
LassoCV Regression: 1363
LassoCV Regression: 1364
LassoCV Regression: 1365
LassoCV Regression: 1366
LassoCV Regression: 1367
LassoCV Regression: 1368
LassoCV Regression: 1369
LassoCV Regression: 1370
LassoCV Regression: 1371
LassoCV Regression: 1372
LassoCV Regression: 1373
LassoCV Regression: 1374
LassoCV Regression: 1375
LassoCV Regression: 1376
LassoCV Regression: 1377
LassoCV Regression: 1378
LassoCV Regression: 1379
LassoCV Regression: 1380
LassoCV Regression: 1381
LassoCV Regression: 1382
LassoCV Regression: 1383
LassoCV Regression: 1384
LassoCV Regression: 1385
LassoCV Regression: 1386
LassoCV Regression: 1387
LassoCV Regression: 1388
LassoCV Regression: 1389
LassoCV Regression: 1390
LassoCV Regression: 1391
LassoCV Regression: 1392
LassoCV Regression: 1393
LassoCV Regression: 1394
LassoCV Regression: 1395
LassoCV Regression: 1396
LassoCV Regression: 1397
LassoCV Regression: 1398


...

HMM fit: 1
HMM fit: 2
HMM fit: 3
HMM fit: 4
HMM fit: 5
HMM fit: 6
HMM fit: 7
HMM fit: 8
HMM fit: 9
HMM fit: 10
HMM fit: 11
HMM fit: 12
HMM fit: 13
HMM fit: 14
HMM fit: 15
HMM fit: 16
HMM fit: 17
HMM fit: 18
HMM fit: 19
HMM fit: 20
HMM fit: 21
HMM fit: 22
HMM fit: 23
HMM fit: 24
HMM fit: 25
HMM fit: 26
HMM fit: 27
HMM fit: 28
HMM fit: 29
HMM fit: 30
HMM fit: 31
HMM fit: 32
HMM fit: 33
HMM fit: 34
HMM fit: 35
HMM fit: 36
HMM fit: 37
HMM fit: 38
HMM fit: 39
HMM fit: 40
HMM fit: 41
HMM fit: 42
HMM fit: 43
HMM fit: 44
HMM fit: 45
HMM fit: 46
HMM fit: 47
HMM fit: 48
HMM fit: 49
HMM fit: 50
HMM fit: 51
HMM fit: 52
HMM fit: 53
HMM fit: 54
HMM fit: 55
HMM fit: 56
HMM fit: 57
HMM fit: 58
HMM fit: 59
HMM fit: 60
HMM fit: 61
HMM fit: 62
HMM fit: 63
HMM fit: 64
HMM fit: 65
HMM fit: 66
HMM fit: 67
HMM fit: 68
HMM fit: 69
HMM fit: 70
HMM fit: 71
HMM fit: 72
HMM fit: 73
HMM fit: 74
HMM fit: 75
HMM fit: 76
HMM fit: 77
HMM fit: 78
HMM fit: 79
HMM fit: 80
HMM fit: 81
HMM fit: 82
HMM fit: 83
HMM fit: 84
H

HMM fit: 641
HMM fit: 642
HMM fit: 643
HMM fit: 644
HMM fit: 645
HMM fit: 646
HMM fit: 647
HMM fit: 648
HMM fit: 649
HMM fit: 650
HMM fit: 651
HMM fit: 652
HMM fit: 653
HMM fit: 654
HMM fit: 655
HMM fit: 656
HMM fit: 657
HMM fit: 658
HMM fit: 659
HMM fit: 660
HMM fit: 661
HMM fit: 662
HMM fit: 663
HMM fit: 664
HMM fit: 665
HMM fit: 666
HMM fit: 667
HMM fit: 668
HMM fit: 669
HMM fit: 670
HMM fit: 671
HMM fit: 672
HMM fit: 673
HMM fit: 674
HMM fit: 675
HMM fit: 676
HMM fit: 677
HMM fit: 678
HMM fit: 679


Some rows of transmat_ have zero sum because no transition from the state was ever observed.


HMM fit: 680


ValueError: rows of transmat_ must sum to 1.0 (got [1. 0.])